<a href="https://colab.research.google.com/github/zh19980811/Automatic-Text-Processing-and-Image-Processing-itmo/blob/main/Task_2_Programmers_EN_students_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Information Retrieval

Let's download the classical data set, i.e. the CRANFIELD text set on aeronautics

In [13]:
! wget -q http://ir.dcs.gla.ac.uk/resources/test_collections/cran/cran.tar.gz
! tar -xvf cran.tar.gz
! rm cran.tar.gz*

cran.all.1400
cran.qry
cranqrel
cranqrel.readme


We take queries only (we will consider queries as documents)

In [14]:
! grep -v "^\." cran.qry > just.qry
! head -3 just.qry

what similarity laws must be obeyed when constructing aeroelastic models
of heated high speed aircraft .
what are the structural and aeroelastic problems associated with flight


We combine  multi-string queries into one

In [15]:
raw_query_data = [line.strip() for line in open("just.qry", "r").readlines()]
query_data = [""]

for query_part in raw_query_data:
  query_data[-1] += query_part + " "
  if query_part.endswith("."):
    query_data.append("")

query_data[:2] #Let's output the couple of documents as an example

['what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft . ',
 'what are the structural and aeroelastic problems associated with flight of high speed aircraft . ']

### Let's make queries to our documents

In [52]:
QUERIES = ['yawed cylinder']

## Boolean retrieval
Let's represent each document as a "bitmask": that is a vector with a dimensionality equal to the vocabulary size, which has 1 at every position if the document contains the corresponding term; and 0 if it does not

In [34]:
# in different versions the answer could also differ, therefore it's important to have the same version
! pip install -q scikit-learn==0.22.2.post1

  Preparing metadata (setup.py) ... done
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 417, in run
    _, build_failures = build(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/wheel_builder.py", line 320, in build
    wheel_file = _build_one(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/wheel_builder.py", line 194, in _build_one
    wheel_path = _build_one_inside_env(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/wheel_builder.py", line 241, in _build_one_inside_env
    wheel_path = build_wheel_legacy(
  File "/usr/local/lib/python3.10/dist-pack

In [35]:
from  sklearn.feature_extraction.text import CountVectorizer

encoder = CountVectorizer(binary=True)
encoded_data = encoder.fit_transform(query_data)
encoded_queries = encoder.transform(QUERIES)
list(encoder.vocabulary_)[:3]

['what', 'similarity', 'laws']

Let's look at the representation of the first sentence

In [36]:
id2term = {idx: term for term, idx in encoder.vocabulary_.items()}
non_zero_values_ids = encoded_data[0].nonzero()[1]

terms = [id2term[idx] for idx in non_zero_values_ids]
terms

['what',
 'similarity',
 'laws',
 'must',
 'be',
 'obeyed',
 'when',
 'constructing',
 'aeroelastic',
 'models',
 'of',
 'heated',
 'high',
 'speed',
 'aircraft']

It's fine.

## Task 0

Now for each query from `QUERIES` let's find the nearest document from `query_data` according to the Jaccard similarity index. There are more effictive solutions to do it, but your task is to realize the algorithm computing the Jaccard index and then apply it to our data.

In [43]:
from numpy.lib.arraysetops import union1d
import numpy as np

def jaccard_sim(vector_a: np.array, vector_b: np.array) -> float:
  """
    Similarity or Jaccard similarity index: the ratio of the intersection cardinality to the union cardinality
  """
  # your code here
  J = np.logical_and(vector_a, vector_b).sum() / np.logical_or(vector_a, vector_b).sum()
  return J

#jaccard_sim(np.array([1, 0, 1, 0, 1]), np.array([0, 1, 1, 1, 1]))
#Check that the function works correctly
assert jaccard_sim(np.array([1, 0, 1, 0, 1]), np.array([0, 1, 1, 1, 1])) == 0.4

## Task 1
Now using the code below find the most similar documents for each query.

In [44]:
for q_id, query in enumerate(encoded_queries):
  # bring to the required datatype
  query = query.todense().A1
  docs = [doc.todense().A1 for doc in encoded_data]
  # calculate the Jaccard index
  id2doc2similarity = [(doc_id, doc, jaccard_sim(query, doc)) for doc_id, doc in enumerate(docs)]
  # sort according to it
  closest = sorted(id2doc2similarity, key=lambda x: x[2], reverse=True)

  print("Q: %s\nFOUND:" % QUERIES[q_id])
  # output 3 most similar documents for each query
  for closest_id, _, sim in closest[:3]:
    print("    %d\t%.2f\t%s" %(closest_id, sim, query_data[closest_id]))

Q: compressibility transformation.
FOUND:
    20	0.13	why does the compressibility transformation fail to correlate the high speed data for helium and air . 
    0	0.00	what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft . 
    1	0.00	what are the structural and aeroelastic problems associated with flight of high speed aircraft . 


We see that some texts intersecting with the query only in insignificant terms have a high Jaccard index (that is our ranking function).

# VSM

Now we are going to do similar calculations, but using tf-idf and cosine distance. To practice we make everything "manually", but in "real life" it's better to use existing effective solutions, e.g., cosine distance from scipy library.

In [60]:
QUERIES = ['yawed cylinder']  # fix

In [61]:
from  sklearn.feature_extraction.text import TfidfVectorizer

# Advice: we highly recommend to check what tf-idf vectorizer
# is able to do, and change its parameters

tfidf_encoder = TfidfVectorizer()
tfidf_encoded_data = tfidf_encoder.fit_transform(query_data)
tfidf_encoded_queries = tfidf_encoder.transform(QUERIES)

list(tfidf_encoder.vocabulary_)[:3]

['what', 'similarity', 'laws']

## Task 2
Realize the cosine distance computation

In [62]:
import numpy as np

def cosine_distance(vector_a: np.array, vector_b: np.array) -> float:
    """
    Cosine distance is 1 minus the ratio of the dot product
    and the product of L2-norm (hint: there are such norms in numpy)
    """

    dot_product = np.dot(vector_a, vector_b)

    norm_a = np.linalg.norm(vector_a, ord=2)
    norm_b = np.linalg.norm(vector_b, ord=2)

    if norm_a == 0 or norm_b == 0:
        return 1
    cosine_similarity = dot_product / (norm_a * norm_b)

    distance = 1 - cosine_similarity
    return distance


assert np.isclose(cosine_distance(np.array([1, 0, 1, 1, 1]), np.array([0, 0, 1, 0, 0])), 0.5)



Now let's find the nearset documents to the query according to the cosine distance between the document vector and the query representation

In [63]:
for q_id, query in enumerate(tfidf_encoded_queries):

  # bring to the required datatype
  query = query.todense().A1
  docs = [doc.todense().A1 for doc in tfidf_encoded_data]
  # Cosine distance
  id2doc2similarity = [(doc_id, doc, cosine_distance(query, doc)) \
                       for doc_id, doc in enumerate(docs)]
  # sort according to it
  closest = sorted(id2doc2similarity, key=lambda x: x[2], reverse=False)

  print("Q: %s\nFOUND:" % QUERIES[q_id])

  for closest_id, _, sim in closest[:3]:
    print("    %d\t%.2f\t%s" %(closest_id, sim, query_data[closest_id]))

Q: yawed cylinder
FOUND:
    60	0.56	does there exist a closed-form expression for the local heat transfer around a yawed cylinder . 
    61	0.75	how far around a cylinder and under what conditions of flow,  if any, is the velocity just outside of the boundary layer a linear function of the distance around the cylinder . 
    213	0.81	what is the effect on cylinder buckling of a circumferential stress system that varies in the axial direction . 
